# 填充和步幅

在所有侧边填充1个像素

In [1]:
import torch
from torch import nn

# 1. 给输入X添加批量和通道维度
# 1: 批量大小(batch size), 1: 通道数(channels), 8,8: 高和宽
def comp_conv2d(conv2d, X):
    # 动态适配
    X = X.reshape((1, 1) + X.shape)
    # 2. 执行卷积操作
    Y = conv2d(X)
    # 3. 移除输出中的批量和通道维度，返回二维结果
    # Y.shape[2:] 表示取Y的第2个维度之后的形状
    return Y.reshape(Y.shape[2:])
# 4. 创建一个1输入通道、1输出通道的3x3卷积层
# padding=1 保证输入输出尺寸相同（8x8 → 8x8）
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
# 5. 创建随机输入数据（8x8矩阵）
X = torch.rand(size=(8, 8))
# 6. 调用函数并查看输出形状
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

填充不同的高度和宽度

In [2]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

将高度和宽度的步幅设置为2

In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

一个稍微复杂的例子

In [4]:
'''
1, 1：输入/输出通道数（都是单通道）
kernel_size=(3, 5)：卷积核大小为3行5列（非常规的正方形）
padding=(0, 1)：高方向不填充，宽方向填充1列
stride=(3, 4)：高方向步幅3，宽方向步幅4
用一个 3×5 的窗口在 8×8 图像上扫描：
高方向：从顶部开始，步长3 → 只能放2次（位置0和3，到位置6时超出边界）
宽方向：左右各填1列0后变成10列，步长4 → 只能放2次（位置0和4）
最终生成了一个 2×2 的特征图。
'''
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

通常，当垂直步幅为$s_{h}$、水平步幅为$s_{w}$时，输出形状为$\left \lfloor (\frac{n_{h}-k_{h}+p_{h}+s_{h}}{s_{h}})\times(\frac{n_{w}-k_{w}+p_{w}+s_{w}}{s_{w}} ) \right \rfloor$
如果我们设置了$p_{h}=k_{h}-1$和$p_{u}=k_{u}－1$，则输出形状将简化为$\left \lfloor (\frac{n_{h}+s_{h}-1}{s_{h}})\times(\frac{n_{w}+s_{w}-1}{s_{w}} ) \right \rfloor $。 更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为$(\frac{n_{h}}{s_{h}})\times(\frac {n_{w}}{s_{w}})$

1. 基础变量定义<br>
$n_{h}$：输入特征图的高度；$n_{w}$：输入特征图的宽度；<br>$k_{h}$：池化窗口的高度；$k_{w}$：池化窗口的宽度；<br>$p_{h}$：高度方向的总填充量（如上下共填充 $p_{h}$行）；$p_{w}$：宽度方向的总填充量（如左右共填充$p_{w}$列）；<br>$s_{h}$：垂直方向步幅；$s_{w}$：水平方向步幅；<br>$o_{h}$：输出特征图的高度；$o_{w}$：输出特征图的宽度。<br>
2. 高度方向输出尺寸$o_{h}$的推导<br>
填充后，输入的有效高度为$n_{h} + p_{h}$（原高度 + 总填充量）。<br>
池化窗口从顶部开始滑动，每次向下移动$s_{h}$步，需满足：窗口底部不超过填充后的有效高度，即：$(k_{h} + (o_{h} - 1)s_{h} \leq n_{h} + p_{h})$<br>
对该不等式求解$o_{h}$：$(o_{h} - 1)s_{h} \leq n_{h} + p_{h} - k_{h}$ <br>
$o_{h} - 1 \leq \frac{n_{h} + p_{h} - k_{h}}{s_{h}}$<br>
$o_{h} \leq \frac{n_{h} + p_{h} - k_{h}}{s_{h}} + 1$<br>
$o_{h} = \left\lfloor \frac{n_{h} - k_{h} + p_{h} + s_{h}}{s_{h}} \right\rfloor$（其中$ \lfloor x \rfloor\ $表示对 x 向下取整）
3. 宽度方向输出尺寸$o_{w}$的推导<br>
同理，填充后输入的有效宽度为$n_{w} + p_{w}$，窗口水平滑动需满足：$k_{w} + (o_{w} - 1)s_{w} \leq n_{w} + p_{w}$<br>
求解 $o_{w}$：$o_{w} = \left\lfloor \frac{n_{w} - k_{w} + p_{w} + s_{w}}{s_w} \right\rfloor$<br>
4. 输出形状的通用公式<br>
结合高度和宽度的推导，输出形状为：$ \left\lfloor \frac{n_{h} - k_{h} + p_{h} + s_{h}}{s_{h}} \right\rfloor \times \left\lfloor \frac{n_{w} - k_{w} + p_{w} + s_{w}}{s_{w}} \right\rfloor$ <br>
5. 特殊情况的简化（当$p_{h} = k_{h} - 1$，$p_{w} = k_{w} - 1$时）<br>
将$p_{h} = k_{h} - 1$代入高度方向公式：$ \frac{n_{h} - k_{h} + (k_{h} - 1) + s_{h}}{s_{h}} = \frac{n_{h} + s_{h} - 1}{s_{h}}$<br>
因此：$o_{h} = \left\lfloor \frac{n_{h} + s_{h} - 1}{s_{h}} \right\rfloor$<br>
同理，宽度方向：$o_{w} = \left\lfloor \frac{n_{w} + s_{w} - 1}{s_{w}} \right\rfloor$<br>
6. 进一步简化（输入尺寸能被步幅整除时）<br>
若$n_{h}$能被$s_{h}$整除（即$n_{h} = s_{h} \times m$)，m 为整数），则：$\left\lfloor \frac{n_{h} + s_{h} - 1}{s_{h}} \right\rfloor = \left\lfloor m + \frac{s_{h} - 1}{s_{h}} \right\rfloor = m = \frac{n_{h}}{s_{h}}$<br>
同理，若$n_{w}$能被$s_{w}$整除，则$o_{w} = \frac{n_{w}}{s_{w}}$。此时输出形状简化为：$\frac{n_{h}}{s_{h}} \times \frac{n_{w}}{s_{w}}$

在二维池化（如最大池化、平均池化）中，输出尺寸的计算公式用于确定池化后特征图的高度和宽度，具体如下：<br>
通用公式对于输入特征图：<br>
高度为 H，宽度为 W；池化窗口（kernel）的高度为$k_{H}$，宽度为$k_{W}$；步长（stride）在高度方向为$s_{H}$，宽度方向为$s_{W}$；填充（padding）在高度方向为$p_{H}$（上下各填充$p_{H}$行，总填充量为$2p_{H}$），宽度方向为$p_{W}$（左右各填充$p_{W}$ 列，总填充量为$2p_{W}$）；则输出特征图的高度$O_{H}$和宽度$O_{W}$为：$O_{H} = \left\lfloor \frac{H + 2p_{H} - k_{H}}{s_{H}} \right\rfloor + 1)$ $O_{W} = \left\lfloor \frac{W + 2p_{W} - k_{W}}{s_{W}} \right\rfloor + 1$<br>
分子$H + 2p_{H} - k_{H}$ 表示：输入高度经填充后，减去窗口高度，得到窗口在垂直方向可移动的总距离；除以步长$s_{H}$ 得到窗口在垂直方向的移动步数；加 1 是因为 “起始位置” 本身也算一个窗口（例如：移动 0 步时对应第一个窗口）。<br>
最终公式：$Output = \frac {Input + 2 \times padding - kernel}{stride} +1$